# This generates a file containing the MJ background estimation from the ABCD method:
## A/B = C/D 
Where,
A = Signal Region (SR)
B = Signal Region Same Sign (SR-SS)
C = Control Region (CR)
D = Control Region Same Sign (CR-SS)

$$\frac{\text{SR}}{\text{SR}_{\text{SS}}} = \frac{\text{CR}}{\text{CR}_{\text{SS}}}$$


$$ \text{SR} = \text{SR}_{\text{SS}} \text{RQCD} $$

### Signal region path$
### Multi jet control region path
### Sample used for Z-QCDjets

In [1]:
import ROOT as r
from ROOT import gStyle
import numpy as np
import ctypes
import os
from IPython.display import display 
import pandas as pd 
from histogramHelpers import *


# Define the path to the channel and the directories for the regions.
channelPath = "/Users/diegomac/Documents/HEP/VBF-Analysis/Zll/High-Mass/"
SR = "SR/"
SRSS = "SR-MJ/"
CR = "SuperCR/"
CRSS = "SuperCR-MJ/"

# Select EWjj and QCDjj samples
EWjj = "Signal_Sherpa"
QCDjj = "Zll_MGRW"

# Remove previous MJ.root file
samples=[i for i in os.listdir(channelPath+SR) if '.root' in i]
print(samples)
if "MJ.root" in samples:
    os.system("rm "+channelPath+SR+"MJ.root")

Welcome to JupyROOT 6.26/10
['MJ.root', 'Zll_MG.root', 'Wjets.root', 'Signal_Sherpa.root', 'Signal_PoPy.root', 'BG.root', 'Zll_MGRW.root', 'Data.root', 'MC.root', 'VV.root', 'Zll_Sherpa.root', 'Zll_SherpaRW.root', 'ttbar.root', 'singletop.root']


In [2]:
histos_no_rebin={
"n_bjets":['n b-jets'],
"lepiso":['Lepton Isolation'],
"n_jets_interval":['N jets gap'],
"delta_R_leplep_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl":['#DeltaR(#mu_{1},#mu_{2})'],
"delta_R_lep1jet_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl":['#DeltaR(#mu_{1},j)'],
"delta_R_lep2jet_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl":['#DeltaR(#mu_{2},j)'],    
}

histos_rebin={
"delta_phi":[[2.0],[0.2,0.8],0.2,'#Delta#phi(#mu_{1},#mu_{2})'],
"lep1_eta_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl":[[0.1],[0.2,0.199],0.2,'#eta(#mu_{1})'],
"lep2_eta_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl":[[0.1],[0.2,0.199],0.2,'#eta(#mu_{2})'],
"ljet0_eta_basic_cuts_ptl":[[-3.0,3.0],[0.5,0.2,0.5],0.2,'#eta(j_{1})'],
"ljet1_eta_basic_cuts_ptl":[[-3.0,3.0],[0.5,0.2,0.5],0.2,'#eta(j_{2})'],  
"lep1_pt":[[100,200,300],[20,50,100,350],20,'pT(#mu_{1})'],
"lep2_pt":[[100,200,300],[20,50,100,350],20,'pT(#mu_{2})'],
"ljet0_pt":[[75,460],[15,35,54],15,'pT(j_{1})'],
"ljet1_pt":[[70,440],[10,37,56],10,'pT(j_{2})'],
"ljet2_pt_basic_cuts_ptl":[[100],[20,50],20,'pT(j_{2})'],
"pt_bal":[[0.15,0.3],[0.03,0.05,0.7],0.15,'pT balance'],
"Z_centrality":[[0.5],[0.1,0.5],0.1,'#xi(Z)'],
"delta_y":[[2.0,6.0],[1.0,0.5,1.0],1.0,'#Deltay_{jj}'],
"inv_mass":[[70,110,140,300],[70,5,10,80,700],5,'m_{#mu#mu}'],
"mass_jj":[[1500],[250,500],250,'m_{jj}'],
"Z_pt_reco_basic_cuts_ptl":[[300,600],[20,50,200],20,'pT(Z)'],
"vec_sum_pt_jets_basic_cuts_ptl":[[300],[20,50],20],
"ratio_zpt_sumjetpt_basic_cuts_ptl":[[0.75,1.25],[0.25,0.1,0.25],0.1],
"met_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl":[[50],[10,50],10,'MET'],
}

histos={}
histos.update(histos_no_rebin)
histos.update(histos_rebin)


In [3]:
mcSamples = [EWjj,QCDjj]
backgroundSamples = ['Wjets','VV',"ttbar",'singletop']
if "Tau" in channelPath:
    backgroundSamples += ['Higgs','Zjets']
mcSamples += backgroundSamples

# Open target file
multiJetFile =r.TFile.Open(channelPath+SR+"MJ.root", "RECREATE")

for histoName in histos:
    # Determine if histogram needs to be rebined
    rebinHistogram = False
    if len(histos[histoName])>2:
        rebinHistogram = True
    
    print(histoName,rebinHistogram)
    
    # Calculate RQCD
    dataSubtractedHistoCR = dataSubtract(histoName,channelPath+CR,"Data",mcSamples,histos,rebin=rebinHistogram)
    dataSubtractedHistoCRSS = dataSubtract(histoName,channelPath+CRSS,"Data",mcSamples,histos,rebin=rebinHistogram)
    makeNegativeBinsZero(dataSubtractedHistoCR)
    makeNegativeBinsZero(dataSubtractedHistoCRSS)
    
    RQCD = dataSubtractedHistoCR.Integral(1,-1)/dataSubtractedHistoCRSS.Integral(1,-1)
    RQCD = 1.0
    print("CR = ",dataSubtractedHistoCR.Integral(1,-1))
    print("CR-SS = ",dataSubtractedHistoCRSS.Integral(1,-1))
    print("RQCD = ",RQCD)
    
    # Calculate the MJ Background shape
    dataSubtractedHistoSRSS = dataSubtract(histoName,channelPath+SRSS,"Data",mcSamples,histos,rebin=rebinHistogram)
    dataSubtractedHistoSRSS.Scale(RQCD)
    makeNegativeBinsZero(dataSubtractedHistoSRSS)
    MJ = dataSubtractedHistoSRSS.Clone()
    
    # Save histogram in MJ.root file
    multiJetFile.WriteObject(MJ,histoName)
    
    
multiJetFile.Close()
    

n_bjets False
CR =  660.0
CR-SS =  20.0
RQCD =  1.0
lepiso False
CR =  547.0
CR-SS =  20.0
RQCD =  1.0
n_jets_interval False
CR =  530.0
CR-SS =  19.0
RQCD =  1.0
delta_R_leplep_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl False
CR =  530.0
CR-SS =  19.0
RQCD =  1.0
delta_R_lep1jet_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl False
CR =  530.0
CR-SS =  19.0
RQCD =  1.0
delta_R_lep2jet_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl False
CR =  530.0
CR-SS =  19.0
RQCD =  1.0
delta_phi True
CR =  75.31969213485718
CR-SS =  11.474272698163986
RQCD =  1.0
lep1_eta_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl True
CR =  84.74600338935852
CR-SS =  14.366638243198395
RQCD =  1.0
lep2_eta_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl True
CR =  71.52938210964203
CR-SS =  13.709694221615791
RQCD =  1.0
ljet0_eta_basic_cuts_ptl True
CR =  87.66579353809357
CR-SS =  14.93

Warning in <TH1F::Rebin>: Bin edge 7 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 7 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 7 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 7 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 7 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 7 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 7 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 7 of r